### Single AWE ANcode Decoder A=83 Synthesizer

In [1]:
import math
import numpy as np

def bits_needed(number):
    return number.bit_length()

In [2]:
GF83_positive = [1, 2, 4, 8, 16, 32, 64, 45, 7, 14, 28, 56, 29, 58, 33, 66, 49, 15, 30, 60, 37, 74, 65, 47, 11, 22, 44, 5, 10, 20, 40, 80, 77, 71, 59, 35, 70, 57, 31, 62, 41] 
GF83_negative = [82, 81, 79, 75, 67, 51, 19, 38, 76, 69, 55, 27, 54, 25, 50, 17, 34, 68, 53, 23, 46, 9, 18, 36, 72, 61, 39, 78, 73, 63, 43, 3, 6, 12, 24, 48, 13, 26, 52, 21, 42]
print(np.size(GF83_positive))
print(np.size(GF83_negative))

41
41


In [3]:
plus_values83 = list(range(1, len(GF83_positive) + 1))
minus_values83 = [-i for i in range(1, len(GF83_negative) + 1)]
print(plus_values83)
print(minus_values83)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41]
[-1, -2, -3, -4, -5, -6, -7, -8, -9, -10, -11, -12, -13, -14, -15, -16, -17, -18, -19, -20, -21, -22, -23, -24, -25, -26, -27, -28, -29, -30, -31, -32, -33, -34, -35, -36, -37, -38, -39, -40, -41]


### 選取特定的位元數:

In [5]:
### 輸入數值為data, ANcode的A為AN
### 乘完A後的數值為 input
data = (1 << 28) - 1        ### 28bits
A = 83
input = data * A
bits = bits_needed(input)

print(input)
print("AN資料的位元數:", bits)
print(bits_needed(A))
print(bits_needed(data))
print(bits_needed(bits))


22280142765
AN資料的位元數: 35
7
28
6


In [6]:
#### SEC_AN Decoder
# 2025/5/12
#-------------------------------------
# Used to do Conventional SEC, corrected errors by AWE
# input: remainder, output: AWE
Bw = bits
Bn = bits_needed(data)
BA = bits_needed(A)
B_location = bits_needed(bits)
#-------------------------------------
fn = 'SECdecoder' +"_AWE_"+ str(Bn) + "bits" +'.v'
f = open(fn, 'w')

f.write('// Product (AN) Code Conventional SECdecoder\n')
f.write('// ' + fn + '\n')
f.write('// Used to do SEC by Galois Field, and corrected AWE\n')
f.write('// Received remainder r, output single AWE.\n')
f.write('module SECdecoder_AWE_%dbits(r, AWE);\n'% (Bn))
f.write('input \t[%d:0]\tr;\n' % (BA-1))
f.write('output\treg\tsigned\t[%d:0]\tAWE;\n' % (np.size(GF83_positive)))

f.write('always@(*) begin\n')
f.write('\tcase(r)\n')
for i in range(np.size(GF83_positive)):
    f.write("\t\t%d: AWE = +(1 << %d);\n" % (GF83_positive[i] ,plus_values83[i]-1))
for j in range(np.size(GF83_negative)):
    f.write("\t\t%d: AWE = -(1 << %d);\n" % (GF83_negative[j] ,plus_values83[j]-1))
f.write("\t\tdefault: AWE = 0;\n")
f.write('\tendcase\n')
f.write('end\n\n')

f.write('endmodule\n')
f.close();
print('Module file %s is generated.\n' % (fn))

Module file SECdecoder_AWE_28bits.v is generated.



In [7]:
#### SEC_AN Decoder
# 2025/5/12
#-------------------------------------
# Used to do Conventional SEC, corrected errors by location, not AWE
# input: remainder, output: location
Bw = bits
Bn = bits_needed(data)
BA = bits_needed(A)
B_location = bits_needed(bits)
#-------------------------------------
fn = 'SECdecoder' +"_location_"+ str(Bn) + "bits" +'.v'
f = open(fn, 'w')

f.write('// Product (AN) Code Conventional SECdecoder\n')
f.write('// ' + fn + '\n')
f.write('// Used to do SEC by Galois Field, and corrected by location, not AWE\n')
f.write('// Received remainder r, output error location.\n')
f.write('module SECdecoder_location_%dbits(r, l);\n'% (Bn))
f.write('input \t[%d:0]\tr;\n' % (BA-1))
f.write('output\treg\tsigned\t[%d:0]\tl;\n' % (B_location))

f.write('always@(*) begin\n')
f.write('\tcase(r)\n')
for i in range(np.size(GF83_positive)):
    f.write("\t\t%d: l = %+d;\n" % (GF83_positive[i] ,plus_values83[i]))
for j in range(np.size(GF83_negative)):
    f.write("\t\t%d: l = %+d;\n" % (GF83_negative[j] ,minus_values83[j]))
f.write("\t\tdefault: l = 0;\n")
f.write('\tendcase\n')
f.write('end\n\n')

f.write('endmodule\n')
f.close();
print('Module file %s is generated.\n' % (fn))

Module file SECdecoder_location_28bits.v is generated.

